In [195]:
import numpy as np
import re
import random
def mapdict(dict,sentence):
    for i in sentence:
        try:
            dict[i]+=1
        except:
            dict[i]=1
    return 
def getcount(word,content):
   
    dict={}
    size=len(content)
    for i in range(size):
            mapdict(dict,content[i])        
    return dict

def addunkown(dictionary,vocab):
    count=0
    keylist=list(dictionary.keys())
    for k in keylist:
        if k in vocab:
            continue
        else:
            count+=dictionary[k]
            del dictionary[k]  
    dictionary['UNK']=count

    return dictionary

def preprocess(content):
     elements=content.strip('\n').split("\t")
     return elements


def maptoken(dictionary,tokens):
    for i in range(len(tokens)):
        try:
            dictionary[tokens[i]]
        except:
            tokens[i]="UNK"
    return tokens


def addksmoothing1(dict,tag_dict,bio_list,word_list,train_word,train_bio,k):
    V=len(word_list)
    for word in word_list:
        for t in bio_list:
            dict[(word,t)]=k/(tag_dict[t]+k*V)
    #map probabilty 
    for i in range(len(train_word)):
        for j in range(len(train_word[i])):
            word=train_word[i][j]
            t=train_bio[i][j]
            dict[(word,t)]+=1/(tag_dict[t]+k*V)
    
    return dict

def addksmoothing2(dict,bio_list,train_bio,k):
#a=Counter(list(zip(*train_bio))[0])
    b=bio_list[:]
    bigram_dict={}
    n_bio=len(b)
    b.append("#")
    for t1 in b:
        for t2 in b[:-1]:
            dict[(t1,t2)]=k
        #map probabilty 
    for i in range(len(train_bio)):
        sequence=["#"]+train_bio[i]
        bigram = [tuple(sequence[i:i+2]) for i in range(len(sequence)-(1))]
        for key in bigram:

            dict[key]+=1
            try:
                bigram_dict[key[0]]+=1
            except:
                bigram_dict[key[0]]=1
    for t1 in b:
        for t2 in b[:-1]:
            dict[(t1,t2)]= dict[(t1,t2)]/(k*n_bio+bigram_dict[t1]) 
    return dict


def evaluate_model_span(predicted_seq, correct_seq):
    T = len(predicted_seq)
    
    num_spans_pred = ['B-' in x for x in predicted_seq].count(True)
    num_spans_ans = ['B-' in x for x in correct_seq].count(True)
    
    correct_pred=0
    
    for i in range(len(correct_seq)):
        
        if 'B-' in correct_seq[i]:
            if correct_seq[i]==predicted_seq[i]:
                flag=1;
                tag_type = correct_seq[i][2:]
                j=i+1
                while( j<T and (correct_seq[j] == 'I-'+tag_type or predicted_seq[j] == 'I-'+tag_type)):

                    if (correct_seq[j]!=predicted_seq[j]):
                        flag=0;
                    
                    j=j+1             
                
                if(flag==1):
                    correct_pred+=1;
                               
                i=j-1
     
    precision = correct_pred/num_spans_pred;
    recall = correct_pred/num_spans_ans;
    
    f_score = (2*precision*recall)/(precision+recall)
                
    return precision, recall, f_score   
def train_test_split(content,coef):
    from sklearn.model_selection import train_test_split
    n=len(content)
    word=content[::3]
    pos=content[1::3]
    bio=content[2::3]
    word=[preprocess(i)for i in word]
    pos=[preprocess(i)for i in pos]
    bio=[preprocess(i)for i in bio]
    word_train, word_test, bio_train, bio_test = train_test_split(word, bio, test_size=1-coef, random_state=42)
    word_train, word_test, pos_train, pos_test = train_test_split(word, pos, test_size=1-coef, random_state=42)
    
    return  word_train,word_test,bio_train,bio_test,pos_train,pos_test




# Split Training data, Pre-process 

In [196]:
filename="train.txt"
f=open(filename,'r',encoding="utf8")
content=f.readlines()
word_train,word_test,bio_train,bio_test,pos_train,pos_test=train_test_split(content,0.9)
word={}
biotag={}
postag={}
word=getcount(word,word_train)
biotag=getcount(biotag,bio_train)
postag=getcount(postag,pos_train)
word_sorted=sorted(word.items(),key=lambda x: (x[1]))
word_sorted=dict(word_sorted)
vocab1=[i for i,x in word_sorted.items() if x>0]
print("total_vocab:")
print(len(vocab1))

k=0
vocab2=np.array([i for i,x in word_sorted.items() if x==1])
t=random.sample(range(len(vocab2)), k)
vocab3=vocab2[t]
d=set(vocab1)-set(vocab3)
vocab=list(d)
print(len(vocab))
#update the dictionary
word_sorted_un={}
word_sorted_un=addunkown(word_sorted,vocab)
#word_train=[maptoken(word_sorted_un,i) for i in word_train]
#apply smoothing
bio_list=list(biotag.keys())
word_list=list(word_sorted_un.keys())
pos_list=list(postag.keys())
#word_test=[maptoken(word_sorted_un,i) for i in word_test]
states=bio_list[:]
states.append("#")

total_vocab:
23031
23031


In [197]:

def featurecount(dict,key1,key2):
    try:
            dict[key1][key2]+=1
    except: 
            try:
                dict[key1][key2]=1
            except:
                dict[key1]={}
                dict[key1][key2]=1
    return 
def getcountdict(dict):
    count_dict={}
    for keys in list(dict.keys()):
            count=sum(dict[keys].values())
            count_dict[keys]=count
    return count_dict

def getaddkprob(key1,key2,dict,count_dict,tag_list,k):
    n=len(tag_list)
    prob=0
    try:
        prob=(dict[key1][key2]+k)/(count_dict[key1]+n*k)
    except:
        try:
            prob=k/(count_dict[key1]+n*k)
        except:
            prob=k/(n*k)
    return prob


#feature 1: tagi/tagi-1,word i
#feature 2: tagi/tagi-1,tagi-2
#feature 3： tagi/word i wordi-1
#feature 4: tagi/postag i, postag i-1
def createfeature1(word_train,bio_train,pos_train,b,w,p):
    dict1={}
    dict2={}
    dict3={}
    dict4={}
    dict5={}
    dict6={}
    dict7={}
    dict8={}
    sum=0
    n_features=8
    for i in range(len(word_train)):
         n=len(word_train[i])
         sum+=n
         tag_sequence=["#"]+bio_train[i]
         pos_sequence=["&"]+pos_train[i]
         word_sequence=["<s>"]+word_train[i]+["</s>"]
         for j in range(n):
                featurecount(dict1,(word_sequence[j+1],tag_sequence[j]),tag_sequence[j+1])
                featurecount(dict2,(word_sequence[j+1],word_sequence[j+2]),tag_sequence[j+1])
                featurecount(dict3,(word_sequence[j],word_sequence[j+1]),tag_sequence[j+1])
                featurecount(dict4,(pos_sequence[j],pos_sequence[j+1]),tag_sequence[j+1])
                featurecount(dict5,(tag_sequence[j]),tag_sequence[j+1])
                featurecount(dict6,(word_sequence[j]),tag_sequence[j+1])
                featurecount(dict7,(word_sequence[j+1]),tag_sequence[j+1])
                featurecount(dict8,(word_sequence[j+2]),tag_sequence[j+1])

    count_dict1=getcountdict(dict1)
    count_dict2=getcountdict(dict2)
    count_dict3=getcountdict(dict3)
    count_dict4=getcountdict(dict4)
    count_dict5=getcountdict(dict5)
    count_dict6=getcountdict(dict6)
    count_dict7=getcountdict(dict7)
    count_dict8=getcountdict(dict8)
    print(sum)
    result=np.zeros((sum,8))
    index=0
    for i in range(len(bio_train)):
        tag_sequence=["#"]+bio_train[i]
        pos_sequence=["&"]+pos_train[i]
        word_sequence=["<s>"]+word_train[i]+["</s>"]
        for j in range(len(bio_train[i])):
            prob1=getaddkprob((word_sequence[j+1],tag_sequence[j]),tag_sequence[j+1],dict1,count_dict1,b,0.01)  #basic feature
            prob2=getaddkprob((word_sequence[j+1],word_sequence[j+2]),tag_sequence[j+1],dict2,count_dict2,w,0.01) #two word
            prob3=getaddkprob((word_sequence[j],word_sequence[j+1]),tag_sequence[j+1],dict3,count_dict3,w,0.01) #two word
            prob4=getaddkprob((pos_sequence[j],pos_sequence[j+1]),tag_sequence[j+1],dict4,count_dict4,p,0.01) # two pos tags
            prob5=getaddkprob((tag_sequence[j]),tag_sequence[j+1],dict5,count_dict5,b,0.01)  #previous tag
            prob6=getaddkprob((word_sequence[j]),tag_sequence[j+1],dict6,count_dict6,w,0.01) #previous word
            prob7=getaddkprob((word_sequence[j+1]),tag_sequence[j+1],dict7,count_dict7,w,0.01) #current word
            prob8=getaddkprob((word_sequence[j+2]),tag_sequence[j+1],dict8,count_dict8,w,0.01) # next word
            result[index]= np.array([prob1,prob2,prob3,prob4,prob5,prob6,prob7,prob8])
            index+=1
    return [dict1,dict2,dict3,dict4,dict5,dict6,dict7,dict8],[count_dict1,count_dict2,count_dict3,count_dict4,count_dict5,count_dict6,count_dict7,count_dict8],result,sum
def mapbio(blist):
    i2b={}
    b2i={}
    for i,b in enumerate(blist):
        b2i[b]=i
        i2b[i]=b
    return i2b,b2i
def mapy(y,dict):
    a=[]
    index=0
    for i in y:
        for j in i:
            a.append(dict[j])
    return a
def mapy2(y,dict):
    a=[]
    index=0
    for i in y:
            a.append(dict[i])
    return a
def mapdict1(dict):
    return dict[0],dict[1],dict[2],dict[3],dict[4],dict[5],dict[6],dict[7]
def ifcap(word):
    if word.title()==word:
        return 1
    else:
        return 0

In [198]:
def read_embedding(filename):
    embed = {}
    for line in open(filename,encoding='utf8'):
        line = line.strip().split()
        if (len(line)!=301):
            continue
        try:
            x = np.array(list(map(float, line[1:])))
            embed[line[0]] = x/np.linalg.norm(x)
        except:
            pass
    
    print('[%s]\n\tEmbedding size: %d' % (filename, len(embed)), end='\n')
    return embed


embed_file="glove.840B.300d.txt"
embed=read_embedding(embed_file)
print("finish loading embedding file")

[glove.840B.300d.txt]
	Embedding size: 2195875
finish loading embedding file


In [199]:
bio_train

[['B-LOC', 'O', 'O', 'O', 'O', 'O', 'O'],
 ['O',
  'O',
  'O',
  'B-PER',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'B-LOC',
  'I-LOC',
  'O'],
 ['B-MISC', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O'],
 ['O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O'],
 ['B-ORG', 'O', 'O', 'O', 'O', 'O'],
 ['O', 'O', 'O', 'O', 'B-MISC', 'I-MISC', 'O'],
 ['B-ORG',
  'O',
  'B-ORG',
  'I-ORG',
  'O',
  'B-ORG',
  'O',
  'B-ORG',
  'O',
  'B-ORG',
  'O',
  'B-ORG',
  'O'],
 ['O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'B-LOC',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O'],
 ['O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O'],
 ['B-ORG', 'O', 'O', 'O', 'O', 'O', 'O', 'O'],
 ['B-ORG', 'O', 'O'

In [200]:
def isNum(s):
    try:
        float(s)
        return 1
    except:
        return 0

# Vectorize

In [201]:
def getembedding(dict,word):
    try:
        embedding=dict[word]
    except:
        try:
            embedding=dict[word.lower()]
        except:
            embedding=np.zeros(300)
    return embedding
    
def vectorize(current_word,previous_tag, current_pos_tag, isStartingWord,list_dict,bio_list,previous_word,next_word):
    from_current_word=list_dict[0]
    embed_dict=list_dict[1]
    d=924
    result=np.zeros(d,dtype=int)
    
    #Features 0-8: P(t_i/w_i)
    if current_word in from_current_word.keys():
        result[0:9]=from_current_word[current_word]
    else:
        result[0:9]=np.zeros(9)
    #Features 9-17: Modelling P(t_i/t_i-1), place 1 at previous_tag
    result[9:18]=np.zeros(9)
    result[bio_list.index(previous_tag)]=1
    #Feature 18: isCaptalized
    result[18]=int(current_word.istitle())
    #Feature 19:  isStartOfSentence
    result[19]= int(isStartingWord)
    #Feature 20: is Number
    result[20] = isNum(current_word)
    #Feature 21: has Hypen 
    result[21] = int('-' in current_word)
    #Feature 22: is Noun POS Tags are in ['NN','NNS','NNP','NNPS']
    result[22] = int(current_pos_tag in ['NN','NNS','NNP','NNPS'])
    #Word Embeddings
    result[23:323]=getembedding(embed_dict,current_word)
    result[323:623]=getembedding(embed_dict,previous_word)
    result[623:923]=getembedding(embed_dict,next_word)
    
    #Feature 923: current_pos_tag is proper noun
    result[923]=int(current_pos_tag in ['NNP','NNPS'])
    
    #Feature 924: previous word is determiner
    #result[924]= int(previous_word in ['a','an','the'])
    
    
    return result

    
def createfeature1(word_train,bio_train,pos_train,b,w,p,embed):
    dict1={}
    dict2={}
    dict3={}
    dict4={}
    dict5={}
    dict6={}
    dict7={}
    dict8={}
    dict9={}
    sum=0
    n_features=309
    for i in range(len(word_train)):
         n=len(word_train[i])
         sum+=n
         tag_sequence=["#"]+bio_train[i]
         #pos_sequence=["&"]+pos_train[i]
         word_sequence=["<s>"]+["<s>"]+word_train[i]+["</s>"]
         for j in range(n):
            featurecount(dict1,word_sequence[j+2],tag_sequence[j+1])
    return dict1,sum
def flatten_nested_lists(nested_lists):
    result = []
    
    for list_ in nested_lists:
        result+=list_
    
    return result

def mapbio(blist):
    i2b={}
    b2i={}
    for i,b in enumerate(blist):
        b2i[b]=i
        i2b[i]=b
    return i2b,b2i
dict1,sum= createfeature1(word_train,bio_train,pos_train,bio_list,word_list,pos_list,embed)
def mapy(y,dict):
    a=[]
    index=0
    for i in y:
        for j in i:
            a.append(dict[j])
    return a

i2b,b2i=mapbio(bio_list)

In [202]:
import operator
max(dict1["Ahold"].items(),key=operator.itemgetter(1))[0]


'B-LOC'

In [203]:
b2i

{'B-LOC': 0,
 'B-MISC': 4,
 'B-ORG': 5,
 'B-PER': 2,
 'I-LOC': 3,
 'I-MISC': 6,
 'I-ORG': 7,
 'I-PER': 8,
 'O': 1}

In [204]:
featuremap={}
for i in range(len(word_train)):
    word_sequence=["<s>"]+["<s>"]+word_train[i]+["</s>"]
    
    for j in range(len(word_train[i])):
        feature=np.zeros(9,dtype=int)
        feature[b2i[max(dict1[word_sequence[j+2]].items(),key=operator.itemgetter(1))[0]]]=1
        featuremap[word_sequence[j+2]]=feature
   
    

In [205]:
featuremap['Ahold']

array([1, 0, 0, 0, 0, 0, 0, 0, 0])

In [206]:

print(states)

['B-LOC', 'O', 'B-PER', 'I-LOC', 'B-MISC', 'B-ORG', 'I-MISC', 'I-ORG', 'I-PER', '#']


In [207]:
bio_list

['B-LOC', 'O', 'B-PER', 'I-LOC', 'B-MISC', 'B-ORG', 'I-MISC', 'I-ORG', 'I-PER']

# Vectorize

In [208]:
X=np.zeros((sum,924),dtype=int)
index=0
for i in range (len(word_train)):
    word_sequence=["<s>"]+["<s>"]+word_train[i]+["</s>"]
    bio_sequence=['#']+bio_train[i]
    pos_sequence=pos_train[i]
    for j in range(len(word_train[i])):
        
        X[index]=vectorize(word_sequence[j+2],bio_sequence[j],pos_sequence[j],j==0,[featuremap,embed],states,word_sequence[j+1],word_sequence[j+3])
        index+=1
        

val=flatten_nested_lists(word_test)
Xval=np.zeros((len(val),924),dtype=int)
index=0
for i in range (len(word_test)):
    word_sequence=["<s>"]+["<s>"]+word_test[i]+["</s>"]
    bio_sequence=['#']+bio_test[i]
    pos_sequence=pos_test[i]
    for j in range(len(word_test[i])):
        
        Xval[index]=vectorize(word_sequence[j+2],bio_sequence[j],pos_sequence[j],j==0,[featuremap,embed],states,word_sequence[j+1],word_sequence[j+3])
        index+=1
        
y=mapy(bio_train,b2i)
yval=mapy(bio_test,b2i)


from sklearn.linear_model import LogisticRegression
clf = LogisticRegression(random_state=0,solver='lbfgs',multi_class='multinomial',max_iter=50,C=0.01).fit(X, y)

In [209]:
dict1

{'Ahold': {'B-LOC': 3, 'B-ORG': 2, 'B-PER': 1, 'O': 1},
 'launches': {'O': 3},
 'Asian': {'B-MISC': 7, 'O': 2},
 'food': {'O': 25},
 'discount': {'O': 5},
 'stores': {'O': 12},
 '.': {'I-LOC': 4, 'I-MISC': 2, 'I-ORG': 6, 'O': 6563},
 'It': {'O': 174},
 'was': {'O': 928},
 'also': {'O': 192},
 'Williams': {'B-ORG': 3, 'B-PER': 6, 'I-PER': 7},
 "'": {'I-MISC': 10, 'I-ORG': 1, 'O': 118},
 'first': {'O': 342},
 'win': {'O': 85},
 'in': {'I-ORG': 6, 'O': 3007},
 'three': {'O': 168},
 'career': {'O': 14},
 'decisions': {'O': 5},
 'against': {'O': 200},
 'San': {'B-LOC': 10, 'B-ORG': 14, 'I-LOC': 1, 'I-ORG': 2},
 'Francisco': {'B-PER': 6, 'I-LOC': 4, 'I-ORG': 5},
 'Israeli': {'B-MISC': 38},
 'officials': {'O': 99},
 'said': {'O': 1652},
 'no': {'I-MISC': 1, 'O': 138},
 'meeting': {'O': 93},
 'had': {'O': 523},
 'yet': {'O': 35},
 'been': {'O': 341},
 'set': {'O': 61},
 'Cash': {'I-PER': 1, 'O': 9},
 'markets': {'O': 34},
 'are': {'O': 302},
 'expected': {'O': 81},
 'to': {'O': 2991},
 'be': {

In [210]:
clf.score(Xval,yval)

0.956356829712822

In [211]:
print(clf.score(Xval,yval))
#evaluate 
correct_seq=[]
for line in bio_test:
    correct_seq+=line
    
predicted_seq=(mapy2(np.array(clf.predict(Xval),dtype=int),i2b))

correct_seq=[]
for line in bio_test:
    correct_seq+=line
evaluate_model_span(predicted_seq, correct_seq)

0.956356829712822


(0.810081466395112, 0.6932461873638345, 0.7471237379666589)

In [ ]:
bio_list

In [ ]:
states

In [ ]:
import math

# Viterbi-MEMM (Log Reg and features)

In [ ]:

def viterbi_memm(line,bio_list,pos_test):
    #print(line)
    N=len(bio_list) #state-graph of len 
    T=len(line) #observation len T
    viterbi=np.zeros((N,T))
    bp = np.zeros((N,T),dtype=int)
    cur_word=line[0]
    cur_pos_tag=pos_test[0]
    model=clf
    line=line+["</s>"]
    for i in range(N):
        xTe = vectorize(cur_word,'#',cur_pos_tag,1,[featuremap,embed],states,"<s>",line[1])
        viterbi[i][0]=math.log(model.predict_proba([xTe])[0][i])
        bp[i][0]=0
    for t in range(1,T):
        for s in range(N):
            prob_to_state=np.zeros(N)
            for j in range (N):
                cur_word = line[t]
                previous_word=line[t-1]
                next_word=line[t+1]
                cur_pos_tag=pos_test[t]
                prev_tag = bio_list[j]
                xTe = vectorize(cur_word,prev_tag,cur_pos_tag,0,[featuremap,embed],states,previous_word,next_word)
                prob_to_state[j]=viterbi[j][t-1]+math.log(model.predict_proba([xTe])[0][s])
            #print("in viterbi %d"%s)
            #print(np.argmax(prob_to_state))
            viterbi[s][t]=np.max(prob_to_state)
            bp[s][t]=np.argmax(prob_to_state)
    path = []
    tag_index = int(viterbi.argmax(axis=0)[T-1])

    for i in reversed(range(T)):     
            path.append(bio_list[tag_index])
            tag_index = bp[tag_index][i]

    path = path[::-1]
    return path


In [ ]:
predicted_seq=[]
for i,line in enumerate(word_test):
    predicted_seq+=viterbi_memm(line,bio_list,pos_test[i])
    #test

In [ ]:
evaluate_model_span(predicted_seq, correct_seq)

# Compute for test set

In [ ]:
filename="train.txt"
f=open(filename,'r',encoding="utf8")
content=f.readlines()
word_train,_,bio_train,_,pos_train,_=train_test_split(content,1)
word={}
biotag={}
postag={}
word=getcount(word,word_train)
biotag=getcount(biotag,bio_train)
postag=getcount(postag,pos_train)
word_sorted=sorted(word.items(),key=lambda x: (x[1]))
word_sorted=dict(word_sorted)
vocab1=[i for i,x in word_sorted.items() if x>0]
print("total_vocab:")
print(len(vocab1))

k=0
vocab2=np.array([i for i,x in word_sorted.items() if x==1])
t=random.sample(range(len(vocab2)), k)
vocab3=vocab2[t]
d=set(vocab1)-set(vocab3)
vocab=list(d)
print(len(vocab))
#update the dictionary
word_sorted_un={}
word_sorted_un=addunkown(word_sorted,vocab)
word_train=[maptoken(word_sorted_un,i) for i in word_train]
#apply smoothing
#bio_list=list(biotag.keys())
word_list=list(word_sorted_un.keys())
pos_list=list(postag.keys())

dict1,sum= createfeature1(word_train,bio_train,pos_train,bio_list,word_list,pos_list,embed)
featuremap={}
for i in range(len(word_train)):
    word_sequence=["<s>"]+["<s>"]+word_train[i]+["</s>"]
    
    for j in range(len(word_train[i])):
        feature=np.zeros(9,dtype=int)
        feature[b2i[max(dict1[word_sequence[j+2]].items(),key=operator.itemgetter(1))[0]]]=1
        featuremap[word_sequence[j+2]]=feature
   
X=np.zeros((sum,924),dtype=int)
index=0
for i in range (len(word_train)):
    word_sequence=["<s>"]+["<s>"]+word_train[i]+["</s>"]
    bio_sequence=['#']+bio_train[i]
    pos_sequence=pos_train[i]
    for j in range(len(word_train[i])):
        
        X[index]=vectorize(word_sequence[j+2],bio_sequence[j],pos_sequence[j],j==0,[featuremap,embed],states,word_sequence[j+1],word_sequence[j+3])
        index+=1
        

#test
filename="test.txt"
f=open(filename,'r',encoding="utf8")
content=f.readlines()
n=len(content)
word=content[::3]
pos=content[1::3]
index=content[2::3]
word_test=[preprocess(i)for i in word]
pos_test=[preprocess(i)for i in pos]
index_test=[preprocess(i)for i in index]       
        

y=mapy(bio_train,b2i)
#yval=mapy(bio_test,b2i)


from sklearn.linear_model import LogisticRegression
clf = LogisticRegression(random_state=0,solver='lbfgs',multi_class='multinomial',max_iter=50,C=0.01).fit(X, y)

predicted_seq=[]
for i,line in enumerate(word_test):
    predicted_seq+=viterbi_memm(line,bio_list,pos_test[i])
    #test


In [141]:
predicted_seq=[]
for i,line in enumerate(word_test):
    predicted_seq+=viterbi_memm(line,bio_list,pos_test[i])
    #test


In [145]:
#not needed but just to get an idea
clf.score(Xval,yval)

0.9751736367666617

In [113]:
from sklearn.linear_model import LogisticRegression
clf = LogisticRegression(random_state=0,solver='lbfgs',multi_class='multinomial',max_iter=50,C=0.01).fit(X, y)

predicted_seq=[]
for i,line in enumerate(word_test):
    predicted_seq+=viterbi_memm(line,bio_list,pos_test[i])
    #test


In [114]:
bio_list

['B-LOC', 'O', 'B-PER', 'I-LOC', 'B-MISC', 'B-ORG', 'I-MISC', 'I-ORG', 'I-PER']

In [115]:
states

['B-LOC',
 'O',
 'B-PER',
 'I-LOC',
 'B-MISC',
 'B-ORG',
 'I-MISC',
 'I-ORG',
 'I-PER',
 '#']

In [58]:
a=viterbi_memm(word_test[0],bio_list,pos_test[0])
a

['O', 'O', 'B-PER', 'I-PER', 'O', 'B-PER', 'I-PER', 'O', 'B-LOC', 'O', 'O']

In [142]:
word_test[0]

['CRICKET',
 '-',
 'POLLOCK',
 'CONCLUDES',
 'WARWICKSHIRE',
 'CAREER',
 'WITH',
 'FLOURISH',
 '.']

In [144]:
predicted_seq

['O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'B-LOC',
 'O',
 'B-LOC',
 'B-MISC',
 'O',
 'O',
 'B-PER',
 'I-PER',
 'O',
 'O',
 'B-ORG',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'B-ORG',
 'O',
 'B-PER',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'B-ORG',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'B-ORG',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'B-ORG',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'B-ORG',
 'O',
 'B-ORG',
 'O',
 'B-ORG',
 'O',
 'B-ORG',
 'O',
 'B-PER',
 'I-PER',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'B-ORG',
 'O',
 'B-ORG',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',

In [146]:
len(flatten_nested_lists(word_test))

44265

In [147]:
len(predicted_seq)

44265

# Write results

In [72]:
from  collections import defaultdict
import csv

In [148]:
#result = { PER : [] }
def gen_tag_indexes(pred):
    result = defaultdict(list)
    T = len(pred)
    for i in range(len(pred)):
        
        if 'B-' in pred[i]:
            flag=1;
            tag_type = pred[i][2:]
            j=i+1
            while j<T and (pred[j] == 'I-'+tag_type) :
                j=j+1             
                
            j=j-1 #To get to last token
            result[tag_type].append(str(i)+'-'+str(j))
            #if i==j:
            #    result[tag_type].append(str(i))       
            #else:
            #    result[tag_type].append(str(i)+'-'+str(j))
    return result

In [149]:
a= gen_tag_indexes(predicted_seq)
a

defaultdict(list,
            {'LOC': ['9-9',
              '11-11',
              '202-202',
              '203-203',
              '205-205',
              '207-207',
              '231-231',
              '254-254',
              '263-263',
              '265-265',
              '296-296',
              '301-301',
              '312-312',
              '326-326',
              '360-360',
              '374-374',
              '434-434',
              '474-474',
              '483-483',
              '492-492',
              '706-706',
              '714-714',
              '773-773',
              '774-775',
              '780-780',
              '794-794',
              '796-796',
              '803-803',
              '820-820',
              '924-924',
              '965-965',
              '1008-1008',
              '1089-1089',
              '1328-1328',
              '1407-1407',
              '1417-1417',
              '1436-1436',
              '1485-1485',
              '14

In [124]:
print(len(flatten_nested_lists(word_test)))
print(len(predicted_seq))

44265
44265


In [125]:
word_test

[['CRICKET',
  '-',
  'POLLOCK',
  'CONCLUDES',
  'WARWICKSHIRE',
  'CAREER',
  'WITH',
  'FLOURISH',
  '.'],
 ['LONDON', '1996-08-25'],
 ['South',
  'African',
  'fast',
  'bowler',
  'Shaun',
  'Pollock',
  'concluded',
  'his',
  'Warwickshire',
  'career',
  'with',
  'a',
  'flourish',
  'on',
  'Sunday',
  'by',
  'taking',
  'the',
  'final',
  'three',
  'wickets',
  'during',
  'the',
  'county',
  "'s",
  'Sunday',
  'league',
  'victory',
  'over',
  'Worcestershire',
  '.'],
 ['Pollock',
  ',',
  'who',
  'returns',
  'home',
  'on',
  'Tuesday',
  'for',
  'an',
  'ankle',
  'operation',
  ',',
  'took',
  'the',
  'last',
  'three',
  'wickets',
  'in',
  'nine',
  'balls',
  'as',
  'Worcestershire',
  'were',
  'dismissed',
  'for',
  '154',
  '.'],
 ['After',
  'an',
  'hour',
  "'s",
  'interruption',
  'for',
  'rain',
  ',',
  'Warwickshire',
  'then',
  'reached',
  'an',
  'adjusted',
  'target',
  'of',
  '109',
  'with',
  '13',
  'balls',
  'to',
  'spare',
  '

In [189]:
Xval.shape

(20301, 933)

In [188]:
len(predicted_seq)

44265

In [126]:
def write_result(filename,preds):
    with open(filename, 'w') as csvfile:
        filewriter = csv.writer(csvfile, delimiter=',',
                                quotechar='|', quoting=csv.QUOTE_MINIMAL)
        filewriter.writerow(['Type','Prediction'])
        
        filewriter.writerow(['ORG',' '.join(preds['ORG'])])
        filewriter.writerow(['MISC',' '.join(preds['MISC'])])
        filewriter.writerow(['PER',' '.join(preds['PER'])])
        filewriter.writerow(['LOC',' '.join(preds['LOC'])])

In [151]:
write_result('memm3.csv', a)

In [232]:
from collections import defaultdict
preds=gen_tag_indexes(predicted_seq)
preds

defaultdict(list,
            {'PER': ['0-0',
              '1-1',
              '2-2',
              '3-3',
              '4-4',
              '5-5',
              '6-6',
              '7-7',
              '8-8',
              '10-10',
              '13-13',
              '14-14',
              '17-17',
              '18-18',
              '19-19',
              '20-20',
              '21-21',
              '22-22',
              '23-23',
              '24-24',
              '25-25',
              '26-26',
              '27-27',
              '28-28',
              '29-29',
              '30-30',
              '31-31',
              '32-32',
              '33-33',
              '34-34',
              '35-35',
              '36-36',
              '37-37',
              '38-38',
              '39-39',
              '40-40',
              '41-42',
              '43-43',
              '44-44',
              '45-45',
              '46-46',
              '47-47',
              '48-48',
    

In [ ]:

#test
filename="test.txt"
f=open(filename,'r',encoding="utf8")
content=f.readlines()
n=len(content)
word=content[::3]
pos=content[1::3]
index=content[2::3]
word_test=[preprocess(i)for i in word]
pos_test=[preprocess(i)for i in pos]
index_test=[preprocess(i)for i in index]
#word_test=[maptoken(word_sorted_un,i) for i in word_test]


In [ ]:
      
def featurecount(dict,key1,key2):
    try:
            dict[key1][key2]+=1
    except: 
            try:
                dict[key1][key2]=1
            except:
                dict[key1]={}
                dict[key1][key2]=1
    return 

# MEMM - Trigram

In [173]:
dim2=933

In [174]:
def vectorize_tri(current_word,previous_tag, current_pos_tag, isStartingWord,list_dict,bio_list,
                  previous_word,next_word, prev2_tag):
    from_current_word=list_dict[0]
    embed_dict=list_dict[1]
    d=dim2
    result=np.zeros(d,dtype=int)
    
    #Features 0-8: P(t_i/w_i)
    if current_word in from_current_word.keys():
        result[0:9]=from_current_word[current_word]
    else:
        result[0:9]=np.zeros(9)
    #Features 9-17: Modelling P(t_i/t_i-1), place 1 at previous_tag
    result[9:18]=np.zeros(9)
    result[bio_list.index(previous_tag)]=1
    #Feature 18: isCaptalized
    result[18]=int(current_word.istitle())
    #Feature 19:  isStartOfSentence
    result[19]= int(isStartingWord)
    #Feature 20: is Number
    result[20] = isNum(current_word)
    #Feature 21: has Hypen 
    result[21] = int('-' in current_word)
    #Feature 22: is Noun POS Tags are in ['NN','NNS','NNP','NNPS']
    result[22] = int(current_pos_tag in ['NN','NNS','NNP','NNPS'])
    #Word Embeddings
    result[23:323]=getembedding(embed_dict,current_word)
    result[323:623]=getembedding(embed_dict,previous_word)
    result[623:923]=getembedding(embed_dict,next_word)
    
    #Feature 923: current_pos_tag is proper noun
    result[923]=int(current_pos_tag in ['NNP','NNPS'])
    
    #Feature 924: previous word is determiner
    #result[924]= int(previous_word in ['a','an','the'])
    
    #Feature 924-932
    result[924:933]=np.zeros(9)
    result[bio_list.index(prev2_tag)]=1
    
    return result

In [175]:

def viterbi_memm_trigram(line,bio_list,pos_test):
    #print(line)
    N=len(bio_list) #state-graph of len 
    T=len(line) #observation len T
    viterbi=np.zeros((N,T))
    bp = np.zeros((N,T),dtype=int)
    cur_word=line[0]
    cur_pos_tag=pos_test[0]
    model=clf
    line=line+["</s>"]
    for i in range(N):
        xTe = vectorize_tri(cur_word,'#',cur_pos_tag,1,[featuremap,embed],states,"<s>",line[1],'#')
        viterbi[i][0]=math.log(model.predict_proba([xTe])[0][i])
        bp[i][0]=0
    for t in range(1,T):
        for s in range(N):
            prob_to_state=np.zeros(N)
            for j in range (N):
                cur_word = line[t]
                previous_word=line[t-1]
                next_word=line[t+1]
                cur_pos_tag=pos_test[t]
                prev_tag = bio_list[j]
                if t==1:
                    prev2_tag='#'
                else:
                    prev2_tag = bio_list[bp[j][t-1]]
                xTe = vectorize_tri(cur_word,prev_tag,cur_pos_tag,0,[featuremap,embed],states,previous_word,next_word,prev2_tag)
                prob_to_state[j]=viterbi[j][t-1]+math.log(model.predict_proba([xTe])[0][s])
            #print("in viterbi %d"%s)
            #print(np.argmax(prob_to_state))
            viterbi[s][t]=np.max(prob_to_state)
            bp[s][t]=np.argmax(prob_to_state)
    path = []
    tag_index = int(viterbi.argmax(axis=0)[T-1])

    for i in reversed(range(T)):     
            path.append(bio_list[tag_index])
            tag_index = bp[tag_index][i]

    path = path[::-1]
    return path


In [176]:
filename="train.txt"
f=open(filename,'r',encoding="utf8")
content=f.readlines()
word_train,word_test,bio_train,bio_test,pos_train,pos_test=train_test_split(content,0.9)
word={}
biotag={}
postag={}
word=getcount(word,word_train)
biotag=getcount(biotag,bio_train)
postag=getcount(postag,pos_train)
word_sorted=sorted(word.items(),key=lambda x: (x[1]))
word_sorted=dict(word_sorted)
vocab1=[i for i,x in word_sorted.items() if x>0]
print("total_vocab:")
print(len(vocab1))

k=0
vocab2=np.array([i for i,x in word_sorted.items() if x==1])
t=random.sample(range(len(vocab2)), k)
vocab3=vocab2[t]
d=set(vocab1)-set(vocab3)
vocab=list(d)
print(len(vocab))
#update the dictionary
word_sorted_un={}
word_sorted_un=addunkown(word_sorted,vocab)
#word_train=[maptoken(word_sorted_un,i) for i in word_train]
#apply smoothing
bio_list=list(biotag.keys())
word_list=list(word_sorted_un.keys())
pos_list=list(postag.keys())
#word_test=[maptoken(word_sorted_un,i) for i in word_test]
states=bio_list[:]
states.append("#")

total_vocab:
23031
23031


In [178]:
X=np.zeros((sum,dim2),dtype=int)
index=0
for i in range (len(word_train)):
    word_sequence=["<s>"]+["<s>"]+word_train[i]+["</s>"]
    bio_sequence=['#']+['#']+bio_train[i]
    pos_sequence=pos_train[i]
    for j in range(len(word_train[i])):
        
        X[index]=vectorize_tri(word_sequence[j+2],bio_sequence[j+1],pos_sequence[j],j==0,[featuremap,embed],states,
                               word_sequence[j+1],word_sequence[j+3],bio_sequence[j])
        index+=1
        

val=flatten_nested_lists(word_test)
Xval=np.zeros((len(val),dim2),dtype=int)
index=0
for i in range (len(word_test)):
    word_sequence=["<s>"]+["<s>"]+word_test[i]+["</s>"]
    bio_sequence=['#']+['#']+bio_test[i]
    pos_sequence=pos_test[i]
    for j in range(len(word_test[i])):
        
        Xval[index]=vectorize_tri(word_sequence[j+2],bio_sequence[j+1],pos_sequence[j],j==0,[featuremap,embed],
                                  states,word_sequence[j+1],word_sequence[j+3],bio_sequence[j])
        index+=1
        
y=mapy(bio_train,b2i)
yval=mapy(bio_test,b2i)


from sklearn.linear_model import LogisticRegression
clf = LogisticRegression(random_state=0,solver='lbfgs',multi_class='multinomial',max_iter=50,C=0.01).fit(X, y)

In [179]:
print(clf.score(Xval,yval))
#evaluate 
correct_seq=[]
for line in bio_test:
    correct_seq+=line
    
predicted_seq=(mapy2(np.array(clf.predict(Xval),dtype=int),i2b))

correct_seq=[]
for line in bio_test:
    correct_seq+=line
evaluate_model_span(predicted_seq, correct_seq)

0.9454706664696321


(0.6687898089171974, 0.6405228758169934, 0.6543512129979967)

In [182]:
filename="train.txt"
f=open(filename,'r',encoding="utf8")
content=f.readlines()
word_train,_,bio_train,_,pos_train,_=train_test_split(content,1)
word={}
biotag={}
postag={}
word=getcount(word,word_train)
biotag=getcount(biotag,bio_train)
postag=getcount(postag,pos_train)
word_sorted=sorted(word.items(),key=lambda x: (x[1]))
word_sorted=dict(word_sorted)
vocab1=[i for i,x in word_sorted.items() if x>0]
print("total_vocab:")
print(len(vocab1))

k=0
vocab2=np.array([i for i,x in word_sorted.items() if x==1])
t=random.sample(range(len(vocab2)), k)
vocab3=vocab2[t]
d=set(vocab1)-set(vocab3)
vocab=list(d)
print(len(vocab))
#update the dictionary
word_sorted_un={}
word_sorted_un=addunkown(word_sorted,vocab)
word_train=[maptoken(word_sorted_un,i) for i in word_train]
#apply smoothing
#bio_list=list(biotag.keys())
word_list=list(word_sorted_un.keys())
pos_list=list(postag.keys())

dict1,sum= createfeature1(word_train,bio_train,pos_train,bio_list,word_list,pos_list,embed)
featuremap={}
for i in range(len(word_train)):
    word_sequence=["<s>"]+["<s>"]+word_train[i]+["</s>"]
    
    for j in range(len(word_train[i])):
        feature=np.zeros(9,dtype=int)
        feature[b2i[max(dict1[word_sequence[j+2]].items(),key=operator.itemgetter(1))[0]]]=1
        featuremap[word_sequence[j+2]]=feature
   
X=np.zeros((sum,dim2),dtype=int)
index=0
for i in range (len(word_train)):
    word_sequence=["<s>"]+["<s>"]+word_train[i]+["</s>"]
    bio_sequence=['#']+['#']+bio_train[i]
    pos_sequence=pos_train[i]
    for j in range(len(word_train[i])):
        
        X[index]=vectorize_tri(word_sequence[j+2],bio_sequence[j+1],pos_sequence[j],j==0,[featuremap,embed],states,
                               word_sequence[j+1],word_sequence[j+3],bio_sequence[j])
        index+=1
        

#test
filename="test.txt"
f=open(filename,'r',encoding="utf8")
content=f.readlines()
n=len(content)
word=content[::3]
pos=content[1::3]
index=content[2::3]
word_test=[preprocess(i)for i in word]
pos_test=[preprocess(i)for i in pos]
index_test=[preprocess(i)for i in index]       
        

y=mapy(bio_train,b2i)
#yval=mapy(bio_test,b2i)


from sklearn.linear_model import LogisticRegression
clf = LogisticRegression(random_state=0,solver='lbfgs',multi_class='multinomial',max_iter=50,C=0.01).fit(X, y)

predicted_seq=[]
for i,line in enumerate(word_test):
    predicted_seq+=viterbi_memm_trigram(line,bio_list,pos_test[i])
    #test


total_vocab:
24339
24339


In [183]:
clf.score(Xval,yval)

0.945519925126841

In [187]:
predicted_seq.count('O')

39032

In [186]:
word_test

[['CRICKET',
  '-',
  'POLLOCK',
  'CONCLUDES',
  'WARWICKSHIRE',
  'CAREER',
  'WITH',
  'FLOURISH',
  '.'],
 ['LONDON', '1996-08-25'],
 ['South',
  'African',
  'fast',
  'bowler',
  'Shaun',
  'Pollock',
  'concluded',
  'his',
  'Warwickshire',
  'career',
  'with',
  'a',
  'flourish',
  'on',
  'Sunday',
  'by',
  'taking',
  'the',
  'final',
  'three',
  'wickets',
  'during',
  'the',
  'county',
  "'s",
  'Sunday',
  'league',
  'victory',
  'over',
  'Worcestershire',
  '.'],
 ['Pollock',
  ',',
  'who',
  'returns',
  'home',
  'on',
  'Tuesday',
  'for',
  'an',
  'ankle',
  'operation',
  ',',
  'took',
  'the',
  'last',
  'three',
  'wickets',
  'in',
  'nine',
  'balls',
  'as',
  'Worcestershire',
  'were',
  'dismissed',
  'for',
  '154',
  '.'],
 ['After',
  'an',
  'hour',
  "'s",
  'interruption',
  'for',
  'rain',
  ',',
  'Warwickshire',
  'then',
  'reached',
  'an',
  'adjusted',
  'target',
  'of',
  '109',
  'with',
  '13',
  'balls',
  'to',
  'spare',
  '